In [ ]:
import picamera
import picamera.array

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import time
import numpy as np
import cv2
from PIL import Image
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input
import cv2
import numpy as np
import os


In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import os
text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)


In [ ]:
def prepare_frame(frame):
    img = Image.fromarray(frame, 'RGB')
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array_extended = np.expand_dims(img_array, axis=0).astype('float32')
    processed = keras.applications.mobilenet.preprocess_input(img_array_extended)
    return processed

In [ ]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
# model.save('my_model.h5')

# # Deletes the existing model
# del model  

# Returns a compiled model identical to the previous one
model = load_model('shape-detection.h5')

In [ ]:
# import paho.mqtt.client as mqtt
# mqttc = mqtt.Client()
# mqttc.connect("mqtt.cmmc.io", 1883, 60)
# mqttc.publish("CMMC/PLUG-001/$/command", "OFF")
# mqttc.publish("CMMC/PLUG-001/$/command", "ON")

# mqttc2 = mqtt.Client()
# mqttc2.connect("localhost", 1883, 60)
# mqttc2.publish("lamp", "on")

In [ ]:
# mqttc2.publish("lamp", "off")

In [ ]:
# mqttc2.publish("lamp", "on")

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import os

# from __future__ import print_function
from imutils.video import VideoStream
from imutils.video import FPS
from picamera.array import PiRGBArray
from picamera import PiCamera
import argparse
import imutils
import time
import cv2
from imutils.video import FPS
import datetime
import time
import numpy as np
import cv2
from PIL import Image

#image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# initialize the camera and stream
camera = PiCamera()
# camera.resolution = (320, 240)
camera.framerate = 32
camera.resolution = (224, 224)
camera.rotation = (270)
camera.start_preview()
# rawCapture = PiRGBArray(camera, size=(320, 240))
rawCapture = PiRGBArray(camera)
stream = camera.capture_continuous(rawCapture, format="bgr",use_video_port=True)

image_widget = widgets.Image()
display(text)
display(image_widget)


for (i, f) in enumerate(stream):
    frame = f.array
    image_widget.value =  bgr8_to_jpeg(frame)
    processed_image = prepare_frame(frame)  # prepare frame
    t1 = datetime.datetime.now()
    prediction = model.predict(processed_image)
    t2 = datetime.datetime.now()
    result = np.argmax(prediction)  # imagenet_utils.decode_predictions(prediction)
    text.value = "class=" + str(result)
#     if str(result) == "0":
#         mqttc2.publish("lamp", "on")
#         mqttc.publish("CMMC/PLUG-001/$/command", "ON")
#         mqttc.publish("CMMC/PLUG-002/$/command", "ON")        
#     else:
#         mqttc2.publish("lamp", "off")        
#         mqttc.publish("CMMC/PLUG-001/$/command", "OFF")
#         mqttc.publish("CMMC/PLUG-002/$/command", "OFF")    
    rawCapture.truncate(0)